# Report of https://arnaud-legrand.shinyapps.io/design_of_experiments/?user_o255

The goal is to find which values maximize the `y` value from 11 different variables.

## Variables dependency

The first thing I attempted to do is trying to know if variables were independant from eachother.

For each pair of variable (55 possibilities), let's do three measurements:
- One is only having "1" for the first variable of the pair
- Second is only having "1" for the second one
- Third is having "1" for both and seeing if it is just the sum of the two transformations

In [ ]:
# Print all triplets useful for the calculations
for i in range(11):
    for j in range(i+1, 11):
        # First has one
        print("0,"*i, "1", ",0"*(10-i), sep="")
        # Second has one
        print("0,"*j, "1", ",0"*(10-j), sep="")
        # Both have one
        print("0,"*i, "1", ",0"*(j-i-1), ",1", ",0"*(10-j), sep="")

Now let's check if the transformations are just the sum of the two individuals, which would mean it is likely independant.

In [2]:
max_diff = 0
with open('user_o255', 'r') as f:
    base_value = None
    for i, l in enumerate(f):
        if not base_value:
            base_value = float(l.split(',')[-1])
            print(f"base_value : {base_value}")
        elif i%3==1:
            first_val = float(l.split(',')[-1])
        elif i%3==2:
            second_val = float(l.split(',')[-1])
        elif i%3==0:
            third_val = float(l.split(',')[-1])
            expected_val = first_val + second_val - base_value
            diff = third_val-expected_val
            if abs(diff)>0.01:
                print(f"Expected {expected_val}, got {third_val} (diff {diff})")
                print(l)
            max_diff = max(abs(diff), max_diff)
print(f"Max diff : {max_diff}")

base_value : 1.01253954454545
Expected 2.01598075043869, got 1.71478938499032 (diff -0.3011913654483702)
0,0,0,1,0,0,1,0,0,0,0,1.71478938499032

Expected 0.023895439375812177, got 0.0114804199592722 (diff -0.012415019416539977)
0,0,0,1,0,0,0,0,1,0,0,0.0114804199592722

Expected -0.9764530904630219, got -0.987702577484058 (diff -0.01124948702103612)
0,0,0,0,0,0,1,0,1,0,0,-0.987702577484058

Max diff : 0.3011913654483702


The maximum delta I had between the expected values from the sum of the two variables and the real value is very low in general, expect for three cases. While two of them could be due to noise, the first one is really high. Let's investigate further.

We are going to try a few values for each variables and try to find how they relate.